Pre-algoritmo

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv

In [2]:
# Matriz M.

# Desde Excel. (No se queda este párrafo)
# Se cargan los datos de la matriz inicial de un archivo Excel.
# Lee el archivo Excel y devuelve un DataFrame de pandas.
# Matriz definida previamente con 5 filas (países) y 15 columnas (productos). 
# df = pd.read_excel('datos.xlsx', header=None)
# Convierte el DataFrame en un array de NumPy, formato matricial.
# M = df.to_numpy()

#c = 10
#p = 100
# Generar una matriz M anidada.
#M = np.zeros((c, p))
#o = 0
#h = int(p/c)+1
#for i in range(c):
#    for j in range(p-o):
#        M[i, j] = 1
#    o = o + h -1

# Cargar la matriz desde el archivo CSV
# Convertimos la matriz M en un array de NumPy.
file_path = 'M_PL_055.csv'
with open(file_path, newline='') as csvfile:
    reader = csv.reader(csvfile)
    M = list(reader)
    M = np.array([row[1:] for row in M[1:]], dtype=float)  # Excluimos la primera fila y el primer elemento de cada fila

# Se obtienen las dimensiones de la matriz.
c, p = M.shape

# Matriz aleatoria. (No se queda este párrafo)
# Se evita la reproducibilidad (datos iniciales distintos cada vez que se ejecute):
# np.random.seed()
# Tamaño de la matriz inicial.
# c = 5 # Número de países/filas.
# p = 5 # Número de productos/columnas.
# peso = 2 # Matriz con valores aleatorios entre 0 y peso-1.
# M = np.random.randint(peso, size=(c, p))

# Si, al final, se quiere representar ln(F), logaritmo = 1, si no, logaritmo = 0.
logaritmo = 1

# (No se queda este párrafo)
#print("Número de países: {}".format(c))
#print("Número de productos: {}".format(p))

Notación

A^{n}: Vector de la variable A en la iteración n.
Ag^{n}: Vector de la variable no normalizada A en la iteración n. (Variable intemedia de iteración).
B_c^{n}: Componente c de la variable B en la iteración n.

Declaración de variables
F: Vector de la Competitividad normalizada.
Q: Vector de la Complejidad normalizadas.
F_valores: Matriz que guarda los valores de F en cada iteración.
Q_valores: Matriz que guarda los valores de Q en cada iteración.

Consideraciones sobre las Condiciones iniciales:
Para el paso previo de calcular Fg^{n} y Qg^{n}, es necesario haber normalizado F^{n-1} y Q^{n-1}. 
Se toma A^{n} = 0 para el caso de Ag^{n-1} = 0.
Esto implica que las condiciones iniciales se imponen sólo para Fg^{0} y Qg^{0} y no se definen para F^{0} y Q^{0}.

Para calcular las variables intermedias Fg^{1} y Qg^{1}, se calculan primero F^{0} y Q^{0}.
F^{0} y Q^{0} están normalizadas, por lo tanto da igual los valores exactos de Fg^{0} y Qg^{0}, es decir, no afectan a la iteración. Sólo afectará si Fg^{0} o Qg^{0} es 0.
Es decir, es idéntico imponer unas condiciones aleatorias a Fg^{0} y Qg^{0} que imponer condiciones aleatorias con valores 0 o 1 a las mismas Fg^{0} y Qg^{0}.

In [8]:
#Condiciones iniciales.
# Se imponen condiciones iniciales para Fg y Qg.
CIartículo = 0
if CIartículo == 1:
    # Valores iniciales Fg y Qg = 1 para todo c y p.
    Fg = np.ones(c)
    Qg = np.ones(p)
else:
    # Valores iniciales de Fg y Qg aleatorios.
    Fg = np.random.randint(2, size=c)
    Qg = np.random.randint(2, size=p)

F = Fg / np.max(Fg)
Q = Qg / np.max(Qg)

# Definimos el número de iteraciones
n = 100

F_valores = np.zeros((n, c))
Q_valores = np.zeros((n, p))

In [10]:
for i in range(n):
    # 1) Cálculo de las variables intermedias:
    # 1.1) Cálculo de Fg en la iteración n:
    Fg = M.dot(Q)  #  donde Fg[c] = \sum_p M_{c,p} * Q[p]

    # 1.2) Cálculo de Qg en la iteración n:
    # vector 1/F
    invF = 1.0 / F
    # Para cada p, sum_c M_{c,p} * (1/F[c]). Usamos la multiplicación de matriz traspuesta y hayamos 1/Q[p].
    invQ = M.T.dot(invF)
    Qg = 1.0 / invQ  # donde Qg[p] = 1 / \sum_c ( M_{c,p} / F_c^{(n-1)} )

    # 3) Guardar los valores de F y Q en cada iteración para su posterior análisis/representación.
    F_valores[i, :] = F
    Q_valores[i, :] = Q

    # 4) Normalización de y actualización de F y Q para la siguiente iteración.
    F = Fg / np.max(Fg)
    Q = Qg / np.max(Qg)

ValueError: could not broadcast input array from shape (64,195) into shape (64,)

In [ ]:
# Graficar la evolución de las componentes de F en función de las iteraciones
plt.figure(figsize=(10, 6))
for j in range(c):
    plt.plot(range(n), F_valores[:, j])
plt.xlabel('Iteraciones')
# ylabel. Si se quiere representar ln(F), se cambia el título del eje y.
if logaritmo == 1:
    plt.ylabel('ln(Competitividad Normalizada (F))')
else:
    plt.ylabel('Competitividad Normalizada (F)')

plt.title('Evolución de las Componentes de F en Función de las Iteraciones')
# Cartel con el tipo de M utilizada.
plt.text(110, logaritmo, f"c = {c}\np = {p}\nNo pesada", bbox=dict(facecolor="white", alpha=0.5))
plt.legend()
plt.grid(True)
plt.show()

# (No se queda este párrafo)
# Exportamos los datos a Excel
# df = pd.DataFrame(F)
# df.to_excel('Resultados.xlsx', sheet_name='Países', index=False)
# df = pd.DataFrame(Q)
# Abre el archivo existente y agrega una nueva hoja
# with pd.ExcelWriter('Resultados.xlsx', engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    # df.to_excel(writer, sheet_name='Productos', index=False)

# Resultados finales
print("Matriz M:")
print(M)
print("\nF (después de {} iteraciones):".format(n))
print(F)
print("\nQ (después de {} iteraciones):".format(n))
print(Q)